# Trade accounting

Performs the Borin and Mancini (2019) exports decomposition. Results are saved as `ta.parquet`, `ta-es.parquet`, and `ta-os.parquet` in `data/interim/trade-accounting/`.

In [7]:
import numpy as np
import pandas as pd
import os
import re
import duckdb
from functions import subset, asvector, zeroout, diagvec, diagmat, diagrow

## Setup

In [2]:
# inputfolder = 'ADB-MRIO'
# outputfileroot = 'ta'
inputfolder = 'ADB-MRIO62'
outputfileroot = 'ta62'
version = None
# version = 'jun2023'

filelist = [file for file in os.listdir(f'../data/interim/{inputfolder}') if not file.startswith('.')]
filelist.sort()

In [3]:
sectors = pd.read_excel('../data/interim/sectors.xlsx')
sectors = sectors.drop_duplicates(subset='ind', ignore_index=True)

# G = 73      # Number of countries + ROW
G = 63      
N = 35      # Number of sectors
f = 5       # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Decompositions

Note that the country index `s` used henceforth corresponds to the MRIO country indices, which start at 1 and not 0.

### Country level

In [4]:
DF = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/interim/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    zuse = np.sum(Z, axis=0)
    zsales = np.sum(Z, axis=1)
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd = zeroout(Y, inverse=True)
    Yf = zeroout(Y)
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    Ad = zeroout(A, inverse=True)
    Af = zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        Exports = np.sum(subset(E, s, -s), axis=0)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = subset(v, s) @ subset(Bnots, s, s)
        VB_FC = subset(v, -s) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        DFs = pd.DataFrame({
            't': year,
            's': s,
            'r': np.setdiff1d(np.arange(1, G+1), s),
            'Exports': Exports,
            'DAVAX1': DAVAX1,
            'DAVAX2': DAVAX2,
            'REX1': REX1,
            'REX2': REX2,
            'REX3': REX3,
            'REF1': REF1,
            'REF2': REF2,
            'FVA': FVA,
            'PDC1': PDC1,
            'PDC2': PDC2
        })
        DF = pd.concat([DF, DFs], ignore_index=True)

    print(f'{year} done')

if version is None:
    outputfilename = f'{outputfileroot}.parquet'
else:
    outputfilename = f'{outputfileroot}_{version}.parquet'

DF.to_parquet(f'../data/interim/trade-accounting/{outputfilename}', index=False)

2000 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done


### By export sectors

In [8]:
DF = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/interim/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    zuse = np.sum(Z, axis=0)
    zsales = np.sum(Z, axis=1)
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd = zeroout(Y, inverse=True)
    Yf = zeroout(Y)
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    Ad = zeroout(A, inverse=True)
    Af = zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        Exports = subset(E, s, -s)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = np.diag(subset(v, s) @ subset(Bnots, s, s))
        VB_FC = np.diag(subset(v, -s) @ subset(Bnots, -s, s))
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = np.diag(subset(v, s) @ subset(Bnots, s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)
        PDC2 = np.diag(subset(v, -s) @ subset(Bnots, -s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)

        DFs = pd.DataFrame({
            't': year,
            's': s,
            'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'i': np.tile(sectors['ind'], G-1),
            'i5': np.tile(sectors['ind5'], G-1),
            'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1),
            'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1),
            'REX2': asvector(REX2),
            'REX3': asvector(REX3),
            'REF1': asvector(REF1),
            'REF2': asvector(REF2),
            'FVA': asvector(FVA),
            'PDC1': asvector(PDC1),
            'PDC2': asvector(PDC2),
        })
        DF = pd.concat([DF, DFs], ignore_index=True)

    print(f'{year} done')

if version is None:
    outputfilename = f'{outputfileroot}-es.parquet'
else:
    outputfilename = f'{outputfileroot}-es_{version}.parquet'

DF.to_parquet(f'../data/interim/trade-accounting/{outputfilename}', index=False)

2000 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done


### By origin sectors

In [9]:
DF = pd.DataFrame()

for file in filelist:
    
    year = re.search('[0-9]{4}', file).group()

    mrio = duckdb.sql(
        f"""
        SELECT * EXCLUDE(C0)
        FROM read_parquet('../data/interim/{inputfolder}/{file}')
        """
    ).df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    zuse = np.sum(Z, axis=0)
    zsales = np.sum(Z, axis=1)
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd = zeroout(Y, inverse=True)
    Yf = zeroout(Y)
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    Ad = zeroout(A, inverse=True)
    Af = zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        Exports = subset(E, s, -s)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = np.diag(subset(v, s)) @ subset(Bnots, s, s)
        VB_FC = diagrow(subset(v, -s)) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        DFs = pd.DataFrame({
            't': year,
            's': s,
            'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'i': np.tile(sectors['ind'], G-1),
            'i5': np.tile(sectors['ind5'], G-1),
            'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1),
            'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1),
            'REX2': asvector(REX2),
            'REX3': asvector(REX3),
            'REF1': asvector(REF1),
            'REF2': asvector(REF2),
            'FVA': asvector(FVA),
            'PDC1': asvector(PDC1),
            'PDC2': asvector(PDC2),
        })
        DF = pd.concat([DF, DFs], ignore_index=True)

    print(f'{year} done')

if version is None:
    outputfilename = f'{outputfileroot}-os.parquet'
else:
    outputfilename = f'{outputfileroot}-os_{version}.parquet'

DF.to_parquet(f'../data/interim/trade-accounting/{outputfilename}', index=False)

2000 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done
